### Required installations

In [1]:
!pip install pythonnet==3.0.0
!pip install git+https://github.com/aiplan4eu/unified-planning@action-selector-mixin
!pip install --pre unified-planning[tamer]
!pip install git+https://github.com/aiplan4eu/up-cpor

  Cloning https://github.com/aiplan4eu/unified-planning (to revision action-selector-mixin) to c:\users\hilak\appdata\local\temp\pip-req-build-_am_2z8y
  Resolved https://github.com/aiplan4eu/unified-planning to commit 0a6fe4420117e6b46f9e50162e846e5a34bb301c
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/aiplan4eu/unified-planning 'C:\Users\hilak\AppData\Local\Temp\pip-req-build-_am_2z8y'
  Running command git checkout -b action-selector-mixin --track origin/action-selector-mixin
  Branch 'action-selector-mixin' set up to track remote branch 'action-selector-mixin' from 'origin'.
  Switched to a new branch 'action-selector-mixin'


  Cloning https://github.com/aiplan4eu/up-cpor to c:\users\hilak\appdata\local\temp\pip-req-build-q4lks4k8
  Resolved https://github.com/aiplan4eu/up-cpor to commit 1de70e9b09e2609fe7fc7bd1cc7ca685b9aad846
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/aiplan4eu/up-cpor 'C:\Users\hilak\AppData\Local\Temp\pip-req-build-q4lks4k8'


### Unified Planning imports ###

In [2]:
from unified_planning.io import PDDLReader
import unified_planning.environment as environment
from unified_planning.engines.results import PlanGenerationResultStatus
from unified_planning.shortcuts import *

## Testing CPOR Engine With Original Solver

We start by defining a simple problem in the UP syntax (alternatively we can use a parser or any other way to create a problem obejct)

In [3]:
# Creating a PDDL reader
reader = PDDLReader()

prob_arr = ['blocks2', 'doors5', 'wumpus05']

In [4]:
for prob in prob_arr:
    print(f"###########################Problem: {prob} start###########################")
    # Parsing a PDDL problem from file
    problem = reader.parse_problem(
        f"../Tests/{prob}/d.pddl",
        f"../Tests/{prob}/p.pddl"
    )

    env = environment.get_environment()
    env.factory.add_engine('CPORPlanning', 'up_cpor.engine', 'CPORImpl')

    with OneshotPlanner(name='CPORPlanning') as planner:
        result = planner.solve(problem)
        if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
            print(f'{planner.name} found a valid plan!')
            print(f'Success')
        else:
            print('No plan found!')

###########################Problem: blocks2 start###########################
NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_environment().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 521 of `C:\Users\hilak\anaconda3\envs\up-cporr\lib\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: CPOR
  * Developers:  Guy Shani
  * Description: CPOR is an offline contingent planner.
  *               It computes a complete plan tree (or graph) where each node is labeled by an action, and edges are labeled by observations.
  *              The leaves of the plan tree correspond to goal states.

CPORPlanning found a valid plan!
Success
###########################Problem: doors5 start###########################
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 521 of `C:\Users\hilak\anaconda3\envs\up-cporr\lib\site-packages\unified_plan

## Testing CPOR Engine With Py-tamer Solver

We start by defining a simple problem in the UP syntax (alternatively we can use a parser or any other way to create a problem obejct)

In [5]:
for prob in prob_arr:
    print(f"###########################Problem: {prob} start###########################")
    # Parsing a PDDL problem from file
    problem = reader.parse_problem(
        f"../Tests/{prob}/d.pddl",
        f"../Tests/{prob}/p.pddl"
    )

    env = environment.get_environment()
    env.factory.add_meta_engine('MetaCPORPlanning', 'up_cpor.engine', 'CPORMetaEngineImpl')

    with OneshotPlanner(name='MetaCPORPlanning[tamer]') as planner:
        result = planner.solve(problem)
        if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
            print(f'{planner.name} found a valid plan!')
            print(f'Success')
        else:
            print('No plan found!')

###########################Problem: blocks2 start###########################
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 521 of `C:\Users\hilak\anaconda3\envs\up-cporr\lib\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: Conitngent Planning Algorithms
  * Developers:  Guy Shani
  * Description: Algorithms for offline and online decision making under partial observability and sensing actions

CPORPlanning[Tamer] found a valid plan!
Success
###########################Problem: doors5 start###########################
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 521 of `C:\Users\hilak\anaconda3\envs\up-cporr\lib\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: Conitngent Planning Algorithms
  * Developers:  Guy Shani
  * Description: Algorithms for offline and online decision making under partial observability and sensing actions

CPO

## Testing SDR Engine With SDR Simulated Env


In [7]:
from up_cpor.simulator import SDRSimulator

for prob in prob_arr:
    print(f"###########################Problem: {prob} start###########################")
    # Parsing a PDDL problem from file
    problem = reader.parse_problem(
        f"../Tests/{prob}/d.pddl",
        f"../Tests/{prob}/p.pddl"
    )

    env = environment.get_environment()
    env.factory.add_engine('SDRPlanning', 'up_cpor.engine', 'SDRImpl')

    with ActionSelector(name='SDRPlanning', problem=problem) as solver:
        simulatedEnv = SDRSimulator(problem)
        while not simulatedEnv.is_goal_reached():
            action = solver.get_action()
            observation = simulatedEnv.apply(action)
            solver.update(observation)

###########################Problem: blocks2 start###########################
  *** Credits ***
  * In operation mode `ActionSelector` at line 711 of `C:\Users\hilak\anaconda3\envs\up-cporr\lib\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: SDR
  * Developers:  Guy Shani
  * Description: SDR is an online contingent replanner.
  *              It provides one action at a time, and then awaits to receive an observation from the environment.

senseclear b1
move-to-t b2 b1
move-t-to-b b1 b2
###########################Problem: doors5 start###########################
  *** Credits ***
  * In operation mode `ActionSelector` at line 711 of `C:\Users\hilak\anaconda3\envs\up-cporr\lib\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: SDR
  * Developers:  Guy Shani
  * Description: SDR is an online contingent replanner.
  *              It provides one action at a time, and then a